In [20]:
import numpy as np
import pandas as pd

# Load dataset
data = pd.read_csv('asiacup.csv')

# Preprocessing
data['Result'] = data['Result'].apply(lambda x: 1 if x.lower() == 'win' else 0)

# Crisp Logic: Use binary win/lose result directly
crisp_y = data['Result'].values.reshape(-1, 1)

# Fuzzy Logic: Introduce a fuzzy version based on heuristics
# Here, a simple heuristic could be that the more runs and fewer wickets lost, the higher the fuzzy result.
data['Fuzzy Result'] = (data['Run Scored'] / data['Run Scored'].max()) * 0.6 + (1 - data['Wicket Lost'] / data['Wicket Lost'].max()) * 0.4
fuzzy_y = data['Fuzzy Result'].values.reshape(-1, 1)

# Features (Run Scored, Wicket Lost, Fours, Sixes, Extras)
X = data[['Run Scored', 'Wicket Lost', 'Fours', 'Sixes', 'Extras']].values

# Handling missing values (NaN)
X = np.nan_to_num(X)
crisp_y = np.nan_to_num(crisp_y)
fuzzy_y = np.nan_to_num(fuzzy_y)

# Check for NaN and Inf values
print("NaN in X:", np.isnan(X).sum(), "Inf in X:", np.isinf(X).sum())
print("NaN in Crisp y:", np.isnan(crisp_y).sum(), "Inf in Crisp y:", np.isinf(crisp_y).sum())
print("NaN in Fuzzy y:", np.isnan(fuzzy_y).sum(), "Inf in Fuzzy y:", np.isinf(fuzzy_y).sum())

# Neural Network parameters
input_neurons = X.shape[1]  # Number of input features
hidden_neurons = 10  # Number of neurons in hidden layer
output_neurons = 1  # Single output (win or lose)
learning_rate = 0.01  # Learning rate
epochs = 10000  # Number of training epochs

# Initialize weights randomly for input-to-hidden and hidden-to-output layers
weights_input_hidden = np.random.uniform(-0.5, 0.5, (input_neurons, hidden_neurons))
weights_hidden_output = np.random.uniform(-0.5, 0.5, (hidden_neurons, output_neurons))

# Sigmoid activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Training process for Crisp or Fuzzy Logic
def train(X, y, logic_type="Crisp"):
    global weights_input_hidden, weights_hidden_output
    
    for epoch in range(epochs):
        # Forward Pass
        hidden_layer_input = np.dot(X, weights_input_hidden)
        hidden_layer_output = sigmoid(hidden_layer_input)
        
        final_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
        predicted_output = sigmoid(final_layer_input)
        
        # Backward Pass
        error = y - predicted_output
        d_predicted_output = error * sigmoid_derivative(predicted_output)
        
        error_hidden_layer = d_predicted_output.dot(weights_hidden_output.T)
        d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
        
        # Update weights
        weights_hidden_output += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
        weights_input_hidden += X.T.dot(d_hidden_layer) * learning_rate
        
        # Print error every 1000 epochs
        if epoch % 1000 == 0:
            total_error = np.mean(np.abs(error))
            print(f"Epoch {epoch} ({logic_type} Logic) Error: {total_error}")

# Call the training function for both Crisp and Fuzzy logic
print("\nTraining for Crisp Logic:")
train(X, crisp_y, logic_type="Crisp")

print("\nTraining for Fuzzy Logic:")
train(X, fuzzy_y, logic_type="Fuzzy")

# Predict function for Crisp or Fuzzy Logic
def predict(X, logic_type="Crisp"):
    hidden_layer_input = np.dot(X, weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    
    final_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    predicted_output = sigmoid(final_layer_input)
    
    # For crisp, we round the prediction to 0 or 1 (binary classification)
    if logic_type == "Crisp":
        return np.where(predicted_output > 0.5, 1, 0)
    # For fuzzy, we keep the continuous value
    else:
        return predicted_output

# Predict and calculate accuracy for Crisp Logic
crisp_predictions = predict(X, logic_type="Crisp")
crisp_accuracy = np.mean(crisp_predictions == crisp_y) * 100
print(f"\nCrisp Logic Accuracy: {crisp_accuracy}%")

# Predict and calculate accuracy for Fuzzy Logic
fuzzy_predictions = predict(X, logic_type="Fuzzy")
fuzzy_accuracy = np.mean(np.abs(fuzzy_predictions - fuzzy_y) < 0.1) * 100  # within 0.1 threshold
print(f"Fuzzy Logic Accuracy: {fuzzy_accuracy}%")


NaN in X: 0 Inf in X: 0
NaN in Crisp y: 0 Inf in Crisp y: 0
NaN in Fuzzy y: 0 Inf in Fuzzy y: 0

Training for Crisp Logic:
Epoch 0 (Crisp Logic) Error: 0.4948521921285138
Epoch 1000 (Crisp Logic) Error: 0.4977641990335058
Epoch 2000 (Crisp Logic) Error: 0.497481014749353
Epoch 3000 (Crisp Logic) Error: 0.4963728554215307
Epoch 4000 (Crisp Logic) Error: 0.49585352831516966
Epoch 5000 (Crisp Logic) Error: 0.49435092922982043
Epoch 6000 (Crisp Logic) Error: 0.49534495654622435
Epoch 7000 (Crisp Logic) Error: 0.4952056315476408
Epoch 8000 (Crisp Logic) Error: 0.4965312813862395
Epoch 9000 (Crisp Logic) Error: 0.49649141758087695

Training for Fuzzy Logic:
Epoch 0 (Fuzzy Logic) Error: 0.13833725902277588


C:\Users\gunde\AppData\Local\Temp\ipykernel_29204\4042997811.py:44: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 1000 (Fuzzy Logic) Error: 0.13296844788482692
Epoch 2000 (Fuzzy Logic) Error: 0.1329485797752414
Epoch 3000 (Fuzzy Logic) Error: 0.13293099386698856
Epoch 4000 (Fuzzy Logic) Error: 0.1329152885917221
Epoch 5000 (Fuzzy Logic) Error: 0.1329011545953307
Epoch 6000 (Fuzzy Logic) Error: 0.1328883490934512
Epoch 7000 (Fuzzy Logic) Error: 0.13287667851412818
Epoch 8000 (Fuzzy Logic) Error: 0.13286598639115552
Epoch 9000 (Fuzzy Logic) Error: 0.13285614471412174

Crisp Logic Accuracy: 51.181102362204726%
Fuzzy Logic Accuracy: 42.125984251968504%
